In [0]:
#6.to write Dataframe as single file with specific name in PySpark

In [0]:
data = [(1,'neeraj kumar','male',2000),(2,'nitin kumar','male',3000)]
schema = ['id','name','gender','salary']

df = spark.createDataFrame(data,schema)
df.show()


+---+------------+------+------+
| id|        name|gender|salary|
+---+------------+------+------+
|  1|neeraj kumar|  male|  2000|
|  2| nitin kumar|  male|  3000|
+---+------------+------+------+



In [0]:
df .write.csv('/FileStore/personsoriginal/',header=True)

In [0]:
spark.read.csv('/FileStore/personsoriginal/',header=True).show()

+---+------------+------+------+
| id|        name|gender|salary|
+---+------------+------+------+
|  1|neeraj kumar|  male|  2000|
|  2| nitin kumar|  male|  3000|
+---+------------+------+------+



In [0]:
#coalesce or #repartition -- to write as a single partition

df.coalesce(1).write.csv('/FileStore/personstemp/',header=True)

In [0]:
dbutils.fs.help()

dbutils.fs provides utilities for working with FileSystems. Most methods in
this package can take either a DBFS path (e.g., "/foo" or "dbfs:/foo"), or
another FileSystem URI.

For more info about a method, use dbutils.fs.help("methodName") .

In notebooks, you can also use the %fs shorthand to access DBFS. The %fs shorthand maps
straightforwardly onto dbutils calls. For example, "%fs head --maxBytes=10000 /file/path"
translates into "dbutils.fs.head("/file/path", maxBytes = 10000)".
 mount mount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Mounts the given source directory into DBFS at the given mount point mounts: Seq -> Displays information about what is mounted within DBFS refreshMounts: boolean -> Forces all machines in this cluster to refresh their mount cache, ensuring they receive the most recent information unmount(mountPoint: String): boolean -> Deletes a DBFS mount point updateMount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Similar to mount(), but updates an existing mount point (if present) instead of creating a new one fsutils cp(from: String, to: String, recurse: boolean = false): boolean -> Copies a file or directory, possibly across FileSystems head(file: String, maxBytes: int = 65536): String -> Returns up to the first 'maxBytes' bytes of the given file as a String encoded in UTF-8 ls(dir: String): Seq -> Lists the contents of a directory mkdirs(dir: String): boolean -> Creates the given directory if it does not exist, also creating any necessary parent directories mv(from: String, to: String, recurse: boolean = false): boolean -> Moves a file or directory, possibly across FileSystems put(file: String, contents: String, overwrite: boolean = false): boolean -> Writes the given String out to a file, encoded in UTF-8 rm(dir: String, recurse: boolean = false): boolean -> Removes a file or directory

In [0]:
filenames = dbutils.fs.ls('/FileStore/personstemp/')
print(filenames)

[FileInfo(path='dbfs:/FileStore/personstemp/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1729323667000), FileInfo(path='dbfs:/FileStore/personstemp/_committed_8010333200259552195', name='_committed_8010333200259552195', size=113, modificationTime=1729323667000), FileInfo(path='dbfs:/FileStore/personstemp/_started_8010333200259552195', name='_started_8010333200259552195', size=0, modificationTime=1729323666000), FileInfo(path='dbfs:/FileStore/personstemp/part-00000-tid-8010333200259552195-bffc5b6d-d714-46ba-9b45-5deea56586d2-158-1-c000.csv', name='part-00000-tid-8010333200259552195-bffc5b6d-d714-46ba-9b45-5deea56586d2-158-1-c000.csv', size=71, modificationTime=1729323667000)]



####1.identify the dataframe data(the single part-file (using coalesce) with specified name)from "personstemp" temporary location

####2. save/stored file into permanent location as a part-file with specific name


####3. remove the temporary location compeletely


In [0]:
#filenames = dbutils.fs.ls('/FileStore/personstemp/')

name=" "

for filename in filenames:
    if filename.name.endswith(".csv"):
        name=filename.name

print(name)

dbutils.fs.cp('/FileStore/personstemp/' + name, '/FileStore/personsdata/persons.csv')



part-00000-tid-8010333200259552195-bffc5b6d-d714-46ba-9b45-5deea56586d2-158-1-c000.csv
Out[47]: True

In [0]:
spark.read.csv("/FileStore/personsdata/persons.csv",header=True).show()

+---+------------+------+------+
| id|        name|gender|salary|
+---+------------+------+------+
|  1|neeraj kumar|  male|  2000|
|  2| nitin kumar|  male|  3000|
+---+------------+------+------+



In [0]:
dbutils.fs.rm("/FileStore/personstemp/", recurse=True) #recursively remove all files & folders from the path

Out[50]: True